In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [7]:
template = cv2.imread('./c159/template.jpeg', 0) # trainImage
cap = cv2.VideoCapture(0)

cv2.namedWindow('image')

# 트랙바 선언
cv2.createTrackbar('Threshold', 'image', 0, 100, lambda x: None)

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

while True:
    
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    threshold = cv2.getTrackbarPos('Threshold','image') / 100.0

    kp1, des1 = sift.detectAndCompute(gray, None)
    kp2, des2 = sift.detectAndCompute(template, None)
    
    
    matches = bf.knnMatch(des1, des2, k=2)
    # Apply ratio test
    good = []
    
    for m,n in matches:
        #print(m.distance, n.distance)
        if m.distance < threshold*n.distance:
            good.append([m])
    
    # cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(gray,kp1,template,kp2,good,None,flags=2)


    # 이미지 출력
    cv2.imshow("image", img3)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(Threshold@image): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


-1

In [13]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1